## Part 1: Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

In [ ]:
# Determine the number of unique values in each column.
attrition_df.nunique()

In [ ]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
y_df.head()


In [ ]:
# Create a list of at least 10 column names to use as X data
selected_columns = ['Age', 'BusinessTravel', 'DistanceFromHome', 'PerformanceRating', 'HourlyRate', 'JobSatisfaction', 'WorkLifeBalance', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'RelationshipSatisfaction']

# Create X_df using your selected columns
X_df = attrition_df[selected_columns]

# Show the data types for X_df
X_df.dtypes



In [ ]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, random_state=78)


In [ ]:
# Convert your X data to numeric data types however you see fit
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)
# Add new code cells as necessary




In [ ]:
# Create a StandardScaler
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
X_scaler = scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
    

In [ ]:
# Create a OneHotEncoder for the Department column
from sklearn.preprocessing import OneHotEncoder
dept_enc = OneHotEncoder(sparse=False)

# Fit the encoder to the training data
department_enc = dept_enc.fit(y_train['Department'].values.reshape(-1, 1))

# Create two new variables by applying the encoder
y_train_dept_enc = department_enc.transform(y_train['Department'].values.reshape(-1, 1))
y_test_dept_enc = department_enc.transform(y_test['Department'].values.reshape(-1, 1))

# to the training and testing data, respectively

In [ ]:
# Create a OneHotEncoder for the Attrition column
att_enc = OneHotEncoder(sparse=False)


# Fit the encoder to the training data
attrition_enc = att_enc.fit(y_train['Attrition'].values.reshape(-1, 1))


# Create two new variables by applying the encoder
y_train_attr_enc = attrition_enc.transform(y_train['Attrition'].values.reshape(-1, 1))
y_test_attr_enc = attrition_enc.transform(y_test['Attrition'].values.reshape(-1, 1))
# to the training and testing data



## Create, Compile, and Train the Model

In [ ]:
# Find the number of columns in the X training data
input_features = len(X_train_scaled[0])

# Create the input layer
inputs = layers.Input(shape=(input_features,))
# Add the first hidden layer
hidden_layer_1 = layers.Dense(units=8, activation='relu')(inputs)
# Add the second hidden layer
hidden_layer_2 = layers.Dense(units=8, activation='relu')(hidden_layer_1)

# Create at least two shared layers
shared_layer_1 = layers.Dense(units=8, activation='relu')(hidden_layer_2)
shared_layer_2 = layers.Dense(units=8, activation='relu')(shared_layer_1)



In [ ]:
# Create a branch for Department
department_branch = layers.Dense(units=8, activation='relu')(shared_layer_2)
department_output = layers.Dense(units=3, activation='softmax', name='Department')(department_branch)

# with a hidden layer and an output layer
# Create the hidden layer
hidden_layer_3 = layers.Dense(units=8, activation='relu')(shared_layer_2)

# Create the output layer
output = layers.Dense(units=2, activation='softmax', name='Department')(hidden_layer_3)

# Create a branch for Attrition
department_branch_branch = layers.Dense(units=8, activation='relu')(shared_layer_2)
department_output = layers.Dense(units=2, activation='softmax', name='Department')(department_branch)



In [ ]:
# Create a branch for Attrition
attrition_branch = layers.Dense(units=8, activation='relu')(shared_layer_2)
attrition_output = layers.Dense(units=2, activation='softmax', name='Attrition')(attrition_branch)

# with a hidden layer and an output layer
# Create the hidden layer
hidden_layer_3 = layers.Dense(units=8, activation='relu')(shared_layer_2)

# Create the output layer
output = layers.Dense(units=2, activation='softmax', name='Attrition')(hidden_layer_3)

# Create a model with the layers



In [ ]:
# Define the output layers
department_hidden = layers.Dense(32, activation='relu', name='department_hidden')(inputs)
department_output = layers.Dense(y_train_dept_enc.shape[1], activation='sigmoid', name='department_output')(department_hidden)

attrition_hidden = layers.Dense(32, activation='relu', name='attrition_hidden')(inputs)   
attrition_output = layers.Dense(y_train_attr_enc.shape[1], activation='sigmoid', name='attrition_output')(attrition_hidden)

# Create the model
model = Model(inputs=inputs, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss = {'department_output': 'categorical_crossentropy',
                    'attrition_output': 'binary_crossentropy'},
              metrics= {
                  'department_output': 'accuracy',
                  'attrition_output': 'accuracy'
              
              })

# Fit the model to the training data



# Summarize the model
model.summary()

In [ ]:
# Train the model
model.fit(X_train_scaled, {'department_output': y_train_dept_enc, 'attrition_output': y_train_attr_enc},
          epochs=100, shuffle=True)


In [ ]:
# Evaluate the model with the testing data
test_results= model.evaluate(X_test_scaled, {'department_output': y_test_dept_enc, 'attrition_output': y_test_attr_enc})
test_results

In [ ]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {test_results[2]}")  
print(f"Attrition Accuracy: {test_results[1]}")


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. The data seems to be inmablanced which means the F one score or balanced accuracy would be better.
2. I have tried relu, sigmoid and softmax. Relu worked for the hidden layers, I used softmax and sigmoid for the output layers.
3. Including more columns, adding more layers, finding more training data or experimenting with activation functions could improve tis model.